In [ ]:
## command to run to infer ProductClassifyTestSetV2

# CUDA_VISIBLE_DEVICES=1,2,5,6 python main_inference_multimodal.py \
#         --model=LLM_Inference_Multimodal \
#         --model.llm_type="clm" \
#         --model.ckpt_path="models/product_title_multitask_multimodal/version_1/epoch=0-step=75000.ckpt" \
#         --model.config_path="models/product_title_multitask_multimodal/version_1/config.yaml" \
#         --model.task="clm_singlemodal_wishtitle2pseudov121tax" \
#         --model.allowed_gen_sequences="datasets/taxonomy/wish_v1.2.1_newtax_leafpaths.txt" \
#         --model.output_dir="models/product_title_multitask_multimodal/version_1" \
#         --model.write_interval="batch" \
#         --data=JSONListData \
#         --data.llm_type="clm" \
#         --data.data_source_yaml_path="datasets/multimodal_multitask/wish_product_classify_testsetv2.yaml" \
#         --data.input_dict="{'template': '{title}', 'task_prefix': 'Generate taxonomy for product: '}" \
#         --data.output_dict="{'template': '{category_path}'}" \
#         --data.transform_dict="{'category_path': 'taxonomy'}" \
#         --data.data_source_type="dvc" \
#         --data.model_name="t5-base" \
#         --data.batch_size=50 \
#         --data.max_length=50 \
#         --data.num_workers=0 \
#         --trainer.logger=false \
#         --trainer.enable_checkpointing=false \
#         --trainer.accelerator gpu \
#         --trainer.strategy ddp

In [2]:
import pandas as pd 
import dvc.api

In [9]:
df_data = pd.read_json(dvc.api.get_url(
    'data/appen_delivery/diamond_testing_validated.json',
    repo='git@github.com:ContextLogic/listing_classification_data.git'
), lines=True)

In [7]:
df_pred = pd.read_json('../../models/product_title_multitask_multimodal/version_1/clm-epoch=0-step=75000--wish_product_classify_testsetv2--test.json', 
    lines=True)

In [8]:
df_pred = df_pred[df_pred.rank_indices == 0].sort_values('batch_indices')

In [10]:
assert len(df_data) == len(df_pred)

In [11]:
from eval_from_file import perf_eval_util

In [18]:
df_data['category'] = df_data['category_path'].apply(lambda x: x.lower().strip().split(' > '))

In [19]:
version = 'mm_clm_v0.1'
df_data[f'{version}_predicted_category'] = df_pred['prediction_decoded'].tolist()
df_data[f'{version}_predicted_category'] = df_data[f'{version}_predicted_category'].apply(lambda x: x.split(' > '))
perm = pd.concat([perf_eval_util(df_data, level=i, col=f'{version}_predicted_category') for i in [1, 2, 0, -1, -2]])

In [21]:
perm[perm['id'] == 'weighted avg']

,id,precision,recall,f1-score,support,level
31,weighted avg,0.867742,0.857683,0.858826,5024.0,1
286,weighted avg,0.769496,0.711584,0.720258,5024.0,2
1979,weighted avg,0.618925,0.539809,0.545154,5024.0,0
580,weighted avg,0.736202,0.668193,0.677423,5024.0,-1
122,weighted avg,0.822690,0.802349,0.806088,5024.0,-2
